In [1]:
import os
import numpy as np
import pandas as pd
import gzip
import requests
import time
from multiprocessing import Pool
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import json
from hashlib import md5
import random
import datetime

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def translator_baidu(text_data):   
    # Set your own appid/appkey.
    appid = 'xxxxxxxxxxxxxxxx'
    appkey = 'xxxxxxxxxxxxxxxxx'

    # For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
    from_lang = 'en'
    to_lang =  'zh'

    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path

    # query = 'Hello World! This is 1st paragraph.\nThis is 2nd paragraph.\nDirect admission from clinic regarding progressive spastic paraparesis with urinary urgency and bowel incontinency.'
    query = str(text_data)

    # Generate salt and sign
    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()

    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)

    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    r = requests.post(url, params=payload, headers=headers)
    result = r.json()
    translated_text = result['trans_result'][0]['dst']

    return translated_text

In [4]:
filename = 'ADMISSIONS.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df_admin = pd.read_csv(file)
df_admin

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1
2,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
3,6,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1
4,7,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26831,39771,32802,128876,2126-01-28 06:51:00,2126-01-30 15:05:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,WHITE,NaN,NaN,NEWBORN,0,1
26832,39772,32803,105824,2118-02-06 17:14:00,2118-06-25 15:00:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,NaN,UNOBTAINABLE,NaN,WHITE,NaN,NaN,NEWBORN,0,1
26833,39777,32806,104049,2162-10-22 03:00:00,2162-11-03 13:50:00,NaN,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,NaN,JEWISH,NaN,WHITE,NaN,NaN,NEWBORN,0,1
26834,39778,32807,140728,2173-10-27 13:00:00,2173-11-02 13:50:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,CAMB,UNOBTAINABLE,MARRIED,UNABLE TO OBTAIN,NaN,NaN,CORONARY ARTERY DISEASE,0,1


In [5]:
df_admin.hospital_expire_flag.value_counts()

hospital_expire_flag
0    23644
1     3192
Name: count, dtype: int64

In [8]:
df_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26836 entries, 0 to 26835
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   row_id                26836 non-null  int64 
 1   subject_id            26836 non-null  int64 
 2   hadm_id               26836 non-null  int64 
 3   admittime             26836 non-null  object
 4   dischtime             26836 non-null  object
 5   deathtime             3192 non-null   object
 6   admission_type        26836 non-null  object
 7   admission_location    26836 non-null  object
 8   discharge_location    26836 non-null  object
 9   insurance             26836 non-null  object
 10  language              4282 non-null   object
 11  religion              26400 non-null  object
 12  marital_status        17781 non-null  object
 13  ethnicity             26836 non-null  object
 14  edregtime             10390 non-null  object
 15  edouttime             10390 non-null

In [9]:
df1 = df_admin[df_admin['hospital_expire_flag']==1]
df1

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
5,8,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,NaN,UNOBTAINABLE,NaN,UNKNOWN/NOT SPECIFIED,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,1
8,11,12,112213,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,Medicare,NaN,JEWISH,MARRIED,WHITE,NaN,NaN,PANCREATIC CANCER/SDA,1,1
14,20,21,111970,2135-01-30 20:50:00,2135-02-08 02:08:00,2135-02-08 02:08:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,JEWISH,MARRIED,WHITE,2135-01-30 18:46:00,2135-01-30 22:05:00,SEPSIS,1,1
22,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,STATUS EPILEPTICUS,1,1
39,55,56,181711,2104-01-02 02:01:00,2104-01-08 10:30:00,2104-01-08 10:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,NOT SPECIFIED,NaN,WHITE,2104-01-01 23:59:00,2104-01-02 03:33:00,HEAD BLEED,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26771,39579,32637,189200,2160-08-26 17:58:00,2160-08-27 08:50:00,2160-08-27 08:50:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicaid,PTUN,CATHOLIC,DIVORCED,WHITE,NaN,NaN,INTRACRANIAL HEMORRHAGE,1,1
26777,39589,32646,183101,2163-12-29 19:59:00,2164-01-01 14:08:00,2164-01-01 14:08:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,ENGL,NOT SPECIFIED,NaN,WHITE,2163-12-29 17:05:00,2163-12-29 20:36:00,S/P MOTOR VEWHICLE ACCIDENT,1,1
26802,39712,32749,138488,2171-10-01 14:33:00,2171-11-19 00:10:00,2171-11-19 00:10:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicaid,ENGL,OTHER,SINGLE,WHITE,NaN,NaN,RESPIRATORY FAILURE,1,1
26812,39723,32763,160695,2124-06-08 18:14:00,2124-07-12 14:45:00,2124-07-12 14:45:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE;CHEST PAIN\CATH,1,1


In [13]:
df_admin1 = df_admin[['subject_id', 'hadm_id', 'admittime', 'dischtime','edregtime', 'edouttime', 'diagnosis', 'hospital_expire_flag']]
df_admin1

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NaN,NEWBORN,0
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0
2,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NaN,NEWBORN,0
3,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NaN,NEWBORN,0
4,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NaN,NEWBORN,0
...,...,...,...,...,...,...,...,...
26831,32802,128876,2126-01-28 06:51:00,2126-01-30 15:05:00,NaN,NaN,NEWBORN,0
26832,32803,105824,2118-02-06 17:14:00,2118-06-25 15:00:00,NaN,NaN,NEWBORN,0
26833,32806,104049,2162-10-22 03:00:00,2162-11-03 13:50:00,NaN,NaN,NEWBORN,0
26834,32807,140728,2173-10-27 13:00:00,2173-11-02 13:50:00,NaN,NaN,CORONARY ARTERY DISEASE,0


In [136]:
filename = 'LABEVENTS.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    temp = pd.read_csv(file)
temp

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag
0,1,2,163353.0,51143,2138-07-17 20:48:00,0,0.00,%,NaN
1,2,2,163353.0,51144,2138-07-17 20:48:00,0,0.00,%,NaN
2,3,2,163353.0,51146,2138-07-17 20:48:00,0,0.00,%,NaN
3,4,2,163353.0,51200,2138-07-17 20:48:00,0,0.00,%,NaN
4,5,2,163353.0,51221,2138-07-17 20:48:00,0,0.00,%,abnormal
...,...,...,...,...,...,...,...,...,...
10896305,24426447,73362,195396.0,51277,2159-04-22 04:30:00,14.9,14.90,%,NaN
10896306,24426448,73362,195396.0,51279,2159-04-22 04:30:00,3.03,3.03,m/uL,abnormal
10896307,24426449,73362,195396.0,51301,2159-04-22 04:30:00,11.3,11.30,K/uL,abnormal
10896308,24426450,73362,195396.0,50919,2159-04-24 04:54:00,HOLD,NaN,NaN,NaN


In [137]:
filename = 'D_LABITEMS.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    temp1 = pd.read_csv(file)
temp1

,row_id,itemid,label,fluid,category,loinc_code
0,1,50800,SPECIMEN TYPE,BLOOD,BLOOD GAS,NaN
1,2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,19991-9
2,3,50802,Base Excess,Blood,Blood Gas,11555-0
3,4,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas,1959-6
4,5,50804,Calculated Total CO2,Blood,Blood Gas,34728-6
...,...,...,...,...,...,...
748,749,51551,VOIDED SPECIMEN,OTHER BODY FLUID,HEMATOLOGY,NaN
749,750,51552,VOIDED SPECIMEN,STOOL,CHEMISTRY,NaN
750,751,51553,VOIDED SPECIMEN,URINE,CHEMISTRY,NaN
751,752,51554,VOIDED SPECIMEN,JOINT FLUID,HEMATOLOGY,NaN


In [14]:
filename = 'CHARTEVENTS.csv.gz'
start = time.time()
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df_chart = pd.read_csv(file)
end = time.time()
print(f"It cost {end-start} seconds!")
df_chart

C:\Users\AaronChen\AppData\Local\Temp\ipykernel_16908\1757400180.py:5: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chart = pd.read_csv(file)


It cost 258.84739780426025 seconds!


,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,3194775,27091,167218,212903,220228,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,13.1,13.1,g/dl,0.0,0.0,NaN,NaN
1,3194776,27091,167218,212903,220545,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,39.4,39.4,%,0.0,0.0,NaN,NaN
2,3194777,27091,167218,212903,220546,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,14.6,14.6,K/uL,1.0,0.0,NaN,NaN
3,3194778,27091,167218,212903,220602,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,101,101.0,mEq/L,0.0,0.0,NaN,NaN
4,3194779,27091,167218,212903,220615,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,.5,0.5,mg/dL,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172484053,304451789,73362,195396,219463,226545,2159-04-20 10:32:00,2159-04-20 10:32:00,20889,White,NaN,NaN,0.0,0.0,NaN,NaN
172484054,304451790,73362,195396,219463,227378,2159-04-20 17:45:00,2159-04-20 17:45:00,20889,CC5B,NaN,NaN,0.0,0.0,NaN,NaN
172484055,304451791,73362,195396,219463,226228,2159-04-21 18:25:00,2159-04-21 18:25:00,20889,M,1.0,NaN,0.0,0.0,NaN,NaN
172484056,304451792,73362,195396,219463,226381,2159-04-21 18:25:00,2159-04-21 18:25:00,20889,Married,2.0,NaN,0.0,0.0,NaN,NaN


In [15]:
filename = 'D_ITEMS.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df_ITEMS = pd.read_csv(file)
df_ITEMS

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,1,1435,Sustained Nystamus,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,2,1436,Tactile Disturbances,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,3,1437,Tremor,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,4,1438,Ulnar Pulse [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,5,1439,Visual Disturbances,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12482,15670,228575,Incision #5- Treatment,Incision #5- Treatment,metavision,chartevents,Skin - Incisions,NaN,Text,NaN
12483,15671,228576,Incision #6- Treatment,Incision #6- Treatment,metavision,chartevents,Skin - Incisions,NaN,Text,NaN
12484,15672,228577,Impaired Skin #2- Type,Impaired Skin #2- Type,metavision,chartevents,Skin - Impairment,NaN,Text,NaN
12485,15673,228578,Impaired Skin #3- Type,Impaired Skin #3- Type,metavision,chartevents,Skin - Impairment,NaN,Text,NaN


In [18]:
df_ITEMS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12487 entries, 0 to 12486
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   row_id        12487 non-null  int64  
 1   itemid        12487 non-null  int64  
 2   label         12483 non-null  object 
 3   abbreviation  2992 non-null   object 
 4   dbsource      12487 non-null  object 
 5   linksto       12487 non-null  object 
 6   category      6049 non-null   object 
 7   unitname      851 non-null    object 
 8   param_type    2992 non-null   object 
 9   conceptid     0 non-null      float64
dtypes: float64(1), int64(2), object(7)
memory usage: 975.7+ KB


In [19]:
df_ITEMS[df_ITEMS['label'].isna()]

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
5346,5427,30140,NaN,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
9059,12247,70001,NaN,NaN,hospital,microbiologyevents,SPECIMEN,NaN,NaN,NaN
9152,12340,80001,NaN,NaN,hospital,microbiologyevents,ORGANISM,NaN,NaN,NaN
9464,12652,90001,NaN,NaN,hospital,microbiologyevents,ANTIBACTERIUM,NaN,NaN,NaN


In [20]:
df_ITEMS.param_type.value_counts()

param_type
Text                1309
Numeric              647
Solution             422
Checkbox             307
Date time            142
Process              125
Numeric with tag      40
Name: count, dtype: int64

In [21]:
df_ITEMS_chartevent = df_ITEMS[df_ITEMS['linksto']=='chartevents']
df_ITEMS_chartevent.to_csv("df_ITEMS_chartevent.csv",index=False)
df_ITEMS_chartevent

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,1,1435,Sustained Nystamus,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,2,1436,Tactile Disturbances,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,3,1437,Tremor,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,4,1438,Ulnar Pulse [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,5,1439,Visual Disturbances,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12482,15670,228575,Incision #5- Treatment,Incision #5- Treatment,metavision,chartevents,Skin - Incisions,NaN,Text,NaN
12483,15671,228576,Incision #6- Treatment,Incision #6- Treatment,metavision,chartevents,Skin - Incisions,NaN,Text,NaN
12484,15672,228577,Impaired Skin #2- Type,Impaired Skin #2- Type,metavision,chartevents,Skin - Impairment,NaN,Text,NaN
12485,15673,228578,Impaired Skin #3- Type,Impaired Skin #3- Type,metavision,chartevents,Skin - Impairment,NaN,Text,NaN


In [22]:
# Step 1: Remove rows where LABEL is empty (assuming empty means NaN or None)
filtered_df = df_ITEMS_chartevent[df_ITEMS_chartevent['label'].notna()]

# Step 2: Find duplicates in the LABEL column
# keep=False marks all duplicates as True, even the first occurrence
duplicates_df = filtered_df[filtered_df.duplicated('label', keep=False)]
duplicates_df = duplicates_df.sort_values(by='label', ascending=True)
duplicates_df

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
9516,12704,220644,ALT,ALT,metavision,chartevents,Labs,NaN,Numeric,NaN
689,690,769,ALT,NaN,carevue,chartevents,Enzymes,NaN,NaN,NaN
32,33,24,ART Lumen Volume,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10138,13326,224404,ART Lumen Volume,ART Lumen Volume,metavision,chartevents,Dialysis,mL,Numeric,NaN
9505,12693,220587,AST,AST,metavision,chartevents,Labs,NaN,Numeric,NaN
...,...,...,...,...,...,...,...,...,...,...
9498,12686,220546,WBC,WBC,metavision,chartevents,Labs,NaN,Numeric,NaN
10045,13233,224672,Yawning,Yawning,metavision,chartevents,Toxicology,NaN,Text,NaN
2850,2851,3695,Yawning,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
979,980,3372,[Color],NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [23]:
duplicates_df.label.value_counts()

label
Daily Wake Up              3
Assistance Device          3
Speech                     3
GI #1 Tube Place Method    3
GI #2 Tube Place Method    3
                          ..
Education Barrier          2
Ectopy Type 2              2
Ectopy Frequency 2         2
ETT Re-taped               2
[Color]                    2
Name: count, Length: 317, dtype: int64

In [25]:
# Load the newly uploaded D_ITEMS.csv file to check its content again after the error
d_items_new = pd.read_csv('df_ITEMS_chartevent.csv')

# Normalize text to improve matching accuracy
normalized_label_new = d_items_new['label'].str.lower()

clinical_features = [
    "age",
    "respiratory rate resp",
    "heart rate",
    "systolic blood pressure sbp",
    "diastolic blood pressure dbp",
    "body temperature",
    "saturation of peripheral oxygen spo",
    "glasgow coma scale - eye",
    "glasgow coma scale - verbal",
    "glasgow coma scale - motor",
    "alanine aminotransferase alt",
    "aspartate aminotransferase ast",
    "albumin",
    "blood urea nitrogen bun",
    "bilirubin",
    "c-reactive protein crp",
    "chloride",
    "creatinine",
    "glucose",
    "hemoglobin",
    "prothrombin time",
    "platelets platelet",
    "potassium",
    "sodium",
    "white blood cell wbc",
    "activated partial thromboplastin time aptt ptt"
]

specific_matches = {} 

# Search for exact matches or close keywords
for feature in clinical_features:
    feature_keywords = feature.split()  # Split feature into keywords to improve matching
    if "glasgow coma scale" in feature:
        # Special handling for Glasgow Coma Scale parts
        gcs_parts = feature.split(" - ")
        specific_matches[feature] = d_items_new[normalized_label_new.str.contains(gcs_parts[1])]
    elif "prothrombin time" in feature:
        # Special handling for INR (common abbreviation for prothrombin time)
        specific_matches[feature] = d_items_new[normalized_label_new.str.contains("inr")]
        
    elif feature == "age":
        # As expected, 'age' might not be in D_ITEMS. We would use demographic data from another table.
        continue
    else:
        search_pattern = '|'.join(feature_keywords)  # Create a regex pattern to match any of the keywords
        specific_matches[feature] = d_items_new[normalized_label_new.str.contains(search_pattern)]

# Create a DataFrame to display the requested table format
matched_features_df = pd.DataFrame([
    {
        "Original Feature": feature,
        "Matched Labels": ", ".join(specific_matches.get(feature, pd.DataFrame())['label'].tolist() if 'label' in specific_matches.get(feature, pd.DataFrame()).columns else [])
    }
    for feature in clinical_features
])

matched_features_df

,Original Feature,Matched Labels
0,age,
1,respiratory rate resp,"Education Response, Epi Infusion Rate, Heart R..."
2,heart rate,"Epi Infusion Rate, Heart Rate, Heart Rhythm, H..."
3,systolic blood pressure sbp,"ABP [Systolic], Arterial BP [Systolic], Arteri..."
4,diastolic blood pressure dbp,"Arterial Pressure, Blood Flow ml/min, Cuff Pre..."
5,body temperature,"RLE [Temperature], RUE [Temperature], Skin [Te..."
6,saturation of peripheral oxygen spo,"Education Response, Height of Bed, Level of Co..."
7,glasgow coma scale - eye,"Eye Care, Eye Opening, Eye Abnormalities, Eye ..."
8,glasgow coma scale - verbal,"Verbal Response, GCS - Verbal Response, GCSVer..."
9,glasgow coma scale - motor,"Motor Deficits, Motor Response, GCS - Motor Re..."


In [80]:
list1 = matched_features_df.iloc[25,1]
list1

'% Inspir. Time, Alarms Activated, Apnea Time Interval, Pause Time %, Plateau Time (7200), Rec.Breath Time(sec), TCPCV Fixed I Time, TCPCV Insp. Time, Bleeding Time, PTT(22-35), APNEA INTERVAL TIME, INS time, time high, time low, INSP TIME, insp time, Kangaroo Care [Time], HFO % I time, Insp Time %, APNEA TIME, Time Disconnect, time disc., APRV Time High, APRV Time low, I-Time, Time, zzzzth in Centimeter, time High, time Lo, I TIME, Time at Breast, ProTime, Ptt, apnea time, PTT, I time, Time High, Time Low, Insp. Time, Time disconnect, apnea time interval, Inspiratory Time, High time, Low time, HFO-% I Time, Insp.Time, Insp time, PAUSE TIME L/R, Insp. time, disconnect time, I time set, Activated Clotting Time, ZProthrombin time, ZPTT, Timeout performed by (CVL), Inspiratory Time, Timeout performed by (PICC), Timeout performed by (PA line), Timeout performed by (PACEN), Timeout performed by (Bronch), Timeout Performed by (LP), Timeout Performed by (THCEN), Dwell Time (Peritoneal Dialysi

In [81]:
list2 = ['PTT(22-35)','Ptt','PTT','ZPTT']
for each in list2:
    if each not in list1.split(", "):
        print(each)

In [28]:
useful_cols = ["Resp Rate (Spont)","Resp Rate (Total)","Respiratory Rate","Spont. Resp. Rate","Resp. Rate","Respiratory Rate (spontaneous)","Respiratory Rate (Total)",
 "Heart Rate",
 "ABP [Systolic]","Arterial BP [Systolic]","Manual BP [Systolic]","NBP [Systolic]","ART BP Systolic","Arterial Blood Pressure systolic","Non Invasive Blood Pressure systolic",
 "ABP [Diastolic]","Arterial BP [Diastolic]","NBP [Diastolic]","BP Cuff [Diastolic]","ART BP Diastolic","Arterial Blood Pressure diastolic","Non Invasive Blood Pressure diastolic",
 "Temperature C","Temperature C (calc)","Temperature F","Temperature F (calc)","TemperatureF_ApacheIV","Temperature Fahrenheit","Temperature Celsius",
 "SpO2","Arterial O2 Saturation","O2 saturation pulseoxymetry",
"Eye Opening","GCS - Eye Opening","GCSEye_ApacheIV","GCSEyeApacheIIValue",
 "Verbal Response","GCSVerbal_ApacheIV","GCSVerbalApacheIIValue (intubated)","GCS - Verbal Response","GCSVerbalApacheIIValue",
 "Motor Response","GCSMotor_ApacheIV","GCS - Motor Response","GCSMotorApacheIIValue",
 'ALT',
 'AST',
 "albumin","Albumin","ALBUMIN",
 "BUN","BUN (6-20)", "bun","BUN    (6-20)","Effluent BUN","BUN_ApacheIV","BunScore_ApacheIV",
 "Bilirubin","Bilirubin_ApacheIV", "Total Bilirubin",
 "C Reactive Protein (CRP)","ZC Reactive Protein (CRP)",
"Chloride (100-112)","Chloride  (100-112)","Chloride (serum)","Chloride (whole blood)",
 "Creatinine (0-1.3)","Creatinine   (0-0.7)", "Creatinine", "Creatinine_ApacheIV", "CreatinineApacheIIScore","CreatinineApacheIIValue",
 "fingerstick glucose","FINGERSTICK GLUCOSE.","Fingerstick Glucose","Glucose (70-105)","Blood Glucose","BloodGlucose", "Glucose","finger stick glucose","Glucose_ApacheIV","GlucoseScore_ApacheIV","Glucose finger stick","Glucose (whole blood)",
 'Hemoglobin',
 'INR (2-4 ref. range)', 'INR', 'ZINR',
 "Platelets","Platelet  (150-440)", "Platelet Count","ZPlatelet Count",
 "Potassium (3.5-5.3)","Potassium  (3.5-5.3)","Potassium","Potassium (whole blood)","PotassiumApacheIIScore","PotassiumApacheIIValue",
 "Sodium (135-148)","Sodium  (135-148)","Sodium", "Sodium_ApacheIV","SodiumScore_ApacheIV","Sodium (whole blood) (soft)", "Sodium (whole blood)", "SodiumApacheIIScore","SodiumApacheIIValue",
 "WBC   (4-11,000)","WBC (4-11,000)","WBC 4.0-11.0","WhiteBloodC 4.0-11.0","WBC","WBC_ApacheIV","WBCScore_ApacheIV","WbcApacheIIScore","WBCApacheIIValue",
 'PTT(22-35)','Ptt','PTT','ZPTT',
"Code Status" 
]
len(useful_cols)

121

In [82]:
df_ITEMS_chartevent_useful = df_ITEMS_chartevent[df_ITEMS_chartevent['label'].isin(useful_cols)].copy()
df_ITEMS_chartevent_useful['name_full'] = df_ITEMS_chartevent_useful['label'] + "~" + df_ITEMS_chartevent_useful['itemid'].astype(str) + "~" + df_ITEMS_chartevent_useful['unitname'].astype(str)

df_ITEMS_chartevent_useful

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid,name_full
13,14,1455,fingerstick glucose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN,fingerstick glucose~1455~nan
31,32,6,ABP [Systolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN,ABP [Systolic]~6~nan
56,57,51,Arterial BP [Systolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN,Arterial BP [Systolic]~51~nan
130,131,128,Code Status,NaN,carevue,chartevents,NaN,NaN,NaN,NaN,Code Status~128~nan
184,185,184,Eye Opening,NaN,carevue,chartevents,NaN,NaN,NaN,NaN,Eye Opening~184~nan
...,...,...,...,...,...,...,...,...,...,...,...
11637,14825,227464,Potassium (whole blood),Potassium (whole blood),metavision,chartevents,Labs,NaN,Numeric with tag,NaN,Potassium (whole blood)~227464~nan
11639,14827,227466,PTT,PTT,metavision,chartevents,Labs,NaN,Numeric with tag,NaN,PTT~227466~nan
11640,14828,227467,INR,INR,metavision,chartevents,Labs,NaN,Numeric with tag,NaN,INR~227467~nan
12117,15305,228112,GCSVerbalApacheIIValue (intubated),GCSVerbalApacheIIValue (intubated),metavision,chartevents,Scores - APACHE II,NaN,Text,NaN,GCSVerbalApacheIIValue (intubated)~228112~nan


In [83]:
temp_sort = df_ITEMS_chartevent_useful[df_ITEMS_chartevent_useful.duplicated('label', keep=False)]
temp_sort.sort_values(by='label', ascending=True)

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid,name_full
689,690,769,ALT,NaN,carevue,chartevents,Enzymes,NaN,NaN,NaN,ALT~769~nan
9516,12704,220644,ALT,ALT,metavision,chartevents,Labs,NaN,Numeric,NaN,ALT~220644~nan
690,691,770,AST,NaN,carevue,chartevents,Enzymes,NaN,NaN,NaN,AST~770~nan
9505,12693,220587,AST,AST,metavision,chartevents,Labs,NaN,Numeric,NaN,AST~220587~nan
11582,14770,227456,Albumin,Albumin,metavision,chartevents,Labs,NaN,Numeric with tag,NaN,Albumin~227456~nan
2972,2973,1521,Albumin,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN,Albumin~1521~nan
1276,1277,1162,BUN,NaN,carevue,chartevents,NaN,NaN,NaN,NaN,BUN~1162~nan
10893,14081,225624,BUN,BUN,metavision,chartevents,Labs,NaN,Numeric,NaN,BUN~225624~nan
130,131,128,Code Status,NaN,carevue,chartevents,NaN,NaN,NaN,NaN,Code Status~128~nan
9566,12754,223758,Code Status,Code Status,metavision,chartevents,General,NaN,Text,NaN,Code Status~223758~nan


In [84]:
df_ITEMS_chartevent_useful1 = df_ITEMS_chartevent_useful.set_index('itemid', inplace=False)
df_fast = df_chart[df_chart['itemid'].isin(df_ITEMS_chartevent_useful1.index)]
df_fast.to_csv("E:/CNSI_project/demo_and_data_5.0/journalCC/MIMIC3_subset/chartevent_filtered.csv", index=False)
df_fast

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,3194775,27091,167218,212903,220228,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,13.1,13.1,g/dl,0.0,0.0,NaN,NaN
2,3194777,27091,167218,212903,220546,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,14.6,14.6,K/uL,1.0,0.0,NaN,NaN
3,3194778,27091,167218,212903,220602,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,101,101.0,mEq/L,0.0,0.0,NaN,NaN
4,3194779,27091,167218,212903,220615,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,.5,0.5,mg/dL,0.0,0.0,NaN,NaN
8,3194783,27091,167218,212903,225624,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,7,7.0,mg/dL,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172483740,255614476,32807,140728,265845,8441,2173-10-29 12:15:00,2173-10-29 12:14:00,17600,61,61.0,mmHg,NaN,NaN,NaN,NotStopd
172483741,255614477,32807,140728,265845,8441,2173-10-29 13:00:00,2173-10-29 14:35:00,17600,74,74.0,mmHg,NaN,NaN,NaN,NotStopd
172483742,255614478,32807,140728,265845,8441,2173-10-29 14:00:00,2173-10-29 14:35:00,17600,73,73.0,mmHg,NaN,NaN,NaN,NotStopd
172483752,255614488,32807,140728,265845,8441,2173-10-29 15:00:00,2173-10-29 15:04:00,17600,86,86.0,mmHg,NaN,NaN,NaN,NotStopd


In [85]:
df_chartevent_filtered = pd.read_csv("chartevent_filtered.csv")
df_chartevent_filtered

C:\Users\AaronChen\AppData\Local\Temp\ipykernel_16908\2174131698.py:2: DtypeWarning: Columns (8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chartevent_filtered = pd.read_csv("E:/CNSI_project/demo_and_data_5.0/journalCC/MIMIC3_subset/chartevent_filtered.csv")


,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,3194775,27091,167218,212903,220228,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,13.1,13.1,g/dl,0.0,0.0,NaN,NaN
1,3194777,27091,167218,212903,220546,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,14.6,14.6,K/uL,1.0,0.0,NaN,NaN
2,3194778,27091,167218,212903,220602,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,101,101.0,mEq/L,0.0,0.0,NaN,NaN
3,3194779,27091,167218,212903,220615,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,.5,0.5,mg/dL,0.0,0.0,NaN,NaN
4,3194783,27091,167218,212903,225624,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,7,7.0,mg/dL,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24324179,255614476,32807,140728,265845,8441,2173-10-29 12:15:00,2173-10-29 12:14:00,17600,61,61.0,mmHg,NaN,NaN,NaN,NotStopd
24324180,255614477,32807,140728,265845,8441,2173-10-29 13:00:00,2173-10-29 14:35:00,17600,74,74.0,mmHg,NaN,NaN,NaN,NotStopd
24324181,255614478,32807,140728,265845,8441,2173-10-29 14:00:00,2173-10-29 14:35:00,17600,73,73.0,mmHg,NaN,NaN,NaN,NotStopd
24324182,255614488,32807,140728,265845,8441,2173-10-29 15:00:00,2173-10-29 15:04:00,17600,86,86.0,mmHg,NaN,NaN,NaN,NotStopd


In [86]:
df_admin1[~df_admin1['hadm_id'].isin(df_chartevent_filtered['hadm_id'])]

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag
49,67,186474,2155-02-25 12:45:00,2155-03-06 15:00:00,NaN,NaN,INCISIONAL HERNIA,0
126,153,191611,2132-09-13 03:34:00,2132-09-15 12:25:00,NaN,NaN,NEWBORN,0
190,227,186539,2155-01-01 21:05:00,2155-01-04 12:30:00,NaN,NaN,NEWBORN,0
198,235,117941,2137-07-08 23:19:00,2137-07-12 13:15:00,2137-07-08 22:53:00,2137-07-08 23:24:00,ST-SEGMENT ELEVATION MYOCARDIAL INFARCTION\CATH,0
241,286,135917,2176-04-10 20:45:00,2176-04-12 16:41:00,NaN,NaN,INFECTED RIGHT FOOT,0
...,...,...,...,...,...,...,...,...
26547,32190,149908,2145-02-22 12:33:00,2145-03-16 15:16:00,NaN,NaN,PREMATURITY,0
26612,32295,184807,2143-01-22 20:50:00,2143-02-09 11:45:00,NaN,NaN,NEWBORN,0
26667,32433,156768,2201-05-07 14:52:00,2201-05-30 13:55:00,NaN,NaN,PREMATURITY,0
26673,32461,112626,2142-03-01 11:12:00,2142-03-05 19:00:00,NaN,NaN,NEWBORN,0


In [87]:
df_admin2 = df_admin1[df_admin1['hadm_id'].isin(df_chartevent_filtered['hadm_id'])]
df_admin2

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NaN,NEWBORN,0
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0
2,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NaN,NEWBORN,0
3,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NaN,NEWBORN,0
4,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NaN,NEWBORN,0
...,...,...,...,...,...,...,...,...
26831,32802,128876,2126-01-28 06:51:00,2126-01-30 15:05:00,NaN,NaN,NEWBORN,0
26832,32803,105824,2118-02-06 17:14:00,2118-06-25 15:00:00,NaN,NaN,NEWBORN,0
26833,32806,104049,2162-10-22 03:00:00,2162-11-03 13:50:00,NaN,NaN,NEWBORN,0
26834,32807,140728,2173-10-27 13:00:00,2173-11-02 13:50:00,NaN,NaN,CORONARY ARTERY DISEASE,0


In [88]:
df_chartevent_filtered1 = df_chartevent_filtered[df_chartevent_filtered['hadm_id'].isin(df_admin2['hadm_id'])]
df_chartevent_filtered1

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,3194775,27091,167218,212903,220228,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,13.1,13.1,g/dl,0.0,0.0,NaN,NaN
1,3194777,27091,167218,212903,220546,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,14.6,14.6,K/uL,1.0,0.0,NaN,NaN
2,3194778,27091,167218,212903,220602,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,101,101.0,mEq/L,0.0,0.0,NaN,NaN
3,3194779,27091,167218,212903,220615,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,.5,0.5,mg/dL,0.0,0.0,NaN,NaN
4,3194783,27091,167218,212903,225624,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,7,7.0,mg/dL,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24324179,255614476,32807,140728,265845,8441,2173-10-29 12:15:00,2173-10-29 12:14:00,17600,61,61.0,mmHg,NaN,NaN,NaN,NotStopd
24324180,255614477,32807,140728,265845,8441,2173-10-29 13:00:00,2173-10-29 14:35:00,17600,74,74.0,mmHg,NaN,NaN,NaN,NotStopd
24324181,255614478,32807,140728,265845,8441,2173-10-29 14:00:00,2173-10-29 14:35:00,17600,73,73.0,mmHg,NaN,NaN,NaN,NotStopd
24324182,255614488,32807,140728,265845,8441,2173-10-29 15:00:00,2173-10-29 15:04:00,17600,86,86.0,mmHg,NaN,NaN,NaN,NotStopd


In [91]:
df_chartevent_filtered2 = df_chartevent_filtered1.copy()
def convert_cell(x):

    try:
        return float(x)
    except (ValueError, TypeError):
        return x

df_chartevent_filtered2['value'] = df_chartevent_filtered2['value'].apply(convert_cell)

df_chartevent_filtered2['valuenum'] = df_chartevent_filtered2['valuenum'].apply(convert_cell)

In [92]:
condition_1 = (df_chartevent_filtered2['value'].apply(pd.to_numeric, errors='coerce').notna()) & \
              (df_chartevent_filtered2['valuenum'].apply(pd.to_numeric, errors='coerce').notna()) & \
              (df_chartevent_filtered2['value'] != df_chartevent_filtered2['valuenum'])

condition_2 = (df_chartevent_filtered2['value'].apply(lambda x: isinstance(x, str))) & \
              (df_chartevent_filtered2['valuenum'].apply(lambda x: isinstance(x, str))) & \
              (df_chartevent_filtered2['value'] != df_chartevent_filtered2['valuenum'])

condition_3 = (df_chartevent_filtered2['value'].isna()) & (df_chartevent_filtered2['valuenum'].notna())


condition_4 = (df_chartevent_filtered2['value'].notna()) & (df_chartevent_filtered2['valuenum'].isna())


condition_5 = df_chartevent_filtered2['valuenum'].apply(lambda x: isinstance(x, str))


condition_6 = df_chartevent_filtered2['value'].apply(lambda x: isinstance(x, str)) & \
              df_chartevent_filtered2['valuenum'].apply(pd.to_numeric, errors='coerce').notna()

condition_7 = df_chartevent_filtered2['value'].isna() & df_chartevent_filtered2['valuenum'].isna()


result_1 = df_chartevent_filtered2[condition_1]
result_2 = df_chartevent_filtered2[condition_2]
result_3 = df_chartevent_filtered2[condition_3]
result_4 = df_chartevent_filtered2[condition_4]
result_5 = df_chartevent_filtered2[condition_5]
result_6 = df_chartevent_filtered2[condition_6]
result_7 = df_chartevent_filtered2[condition_7]


print(f"Condition 1 (numeric columns, unequal values): {len(result_1)} rows")
print(f"Condition 2 (string columns, unequal values): {len(result_2)} rows")
print(f"Condition 3 (value is NaN, valuenum is not NaN): {len(result_3)} rows")
print(f"Condition 4 (value is not NaN, valuenum is NaN): {len(result_4)} rows")
print(f"Condition 5 (valuenum is string): {len(result_5)} rows")
print(f"Condition 6 (value is string, valuenum is numeric): {len(result_6)} rows")
print(f"Condition 7 (both columns are NaN): {len(result_7)} rows")

Condition 1 (numeric columns, unequal values): 558971 rows
Condition 2 (string columns, unequal values): 0 rows
Condition 3 (value is NaN, valuenum is not NaN): 84412 rows
Condition 4 (value is not NaN, valuenum is NaN): 2342379 rows
Condition 5 (valuenum is string): 0 rows
Condition 6 (value is string, valuenum is numeric): 2060456 rows
Condition 7 (both columns are NaN): 39178 rows


In [93]:
def choose_value(row):

    val_value = row['value']
    val_valuenum = row['valuenum']

    if pd.isna(val_value) and pd.isna(val_valuenum):
        return np.nan

    elif not pd.isna(val_value) and not pd.isna(val_valuenum):
        return val_valuenum

    elif not pd.isna(val_value):
        return val_value

    else:
        return val_valuenum

df_chartevent_filtered2['valuenum1'] = df_chartevent_filtered2.apply(choose_value, axis=1)
df_chartevent_filtered2

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped,valuenum1
0,3194775,27091,167218,212903,220228,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,13.1,13.1,g/dl,0.0,0.0,NaN,NaN,13.1
1,3194777,27091,167218,212903,220546,2130-06-10 02:23:00,2130-06-10 03:54:00,20889,14.6,14.6,K/uL,1.0,0.0,NaN,NaN,14.6
2,3194778,27091,167218,212903,220602,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,101.0,101.0,mEq/L,0.0,0.0,NaN,NaN,101.0
3,3194779,27091,167218,212903,220615,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,0.5,0.5,mg/dL,0.0,0.0,NaN,NaN,0.5
4,3194783,27091,167218,212903,225624,2130-06-10 02:23:00,2130-06-10 04:06:00,20889,7.0,7.0,mg/dL,0.0,0.0,NaN,NaN,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24324179,255614476,32807,140728,265845,8441,2173-10-29 12:15:00,2173-10-29 12:14:00,17600,61.0,61.0,mmHg,NaN,NaN,NaN,NotStopd,61.0
24324180,255614477,32807,140728,265845,8441,2173-10-29 13:00:00,2173-10-29 14:35:00,17600,74.0,74.0,mmHg,NaN,NaN,NaN,NotStopd,74.0
24324181,255614478,32807,140728,265845,8441,2173-10-29 14:00:00,2173-10-29 14:35:00,17600,73.0,73.0,mmHg,NaN,NaN,NaN,NotStopd,73.0
24324182,255614488,32807,140728,265845,8441,2173-10-29 15:00:00,2173-10-29 15:04:00,17600,86.0,86.0,mmHg,NaN,NaN,NaN,NotStopd,86.0


In [98]:
filename = 'PATIENTS.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df_patient = pd.read_csv(file)
df_patient

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,1,2,M,2138-07-17 00:00:00,NaN,NaN,NaN,0
1,2,3,M,2025-04-11 00:00:00,2102-06-14 00:00:00,NaN,2102-06-14 00:00:00,1
2,4,5,M,2103-02-02 00:00:00,NaN,NaN,NaN,0
3,6,7,F,2121-05-23 00:00:00,NaN,NaN,NaN,0
4,7,8,M,2117-11-20 00:00:00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
23687,30793,32802,F,2126-01-28 00:00:00,NaN,NaN,NaN,0
23688,30794,32803,F,2118-02-06 00:00:00,NaN,NaN,NaN,0
23689,30797,32806,M,2162-10-22 00:00:00,NaN,NaN,NaN,0
23690,30798,32807,M,2092-10-22 00:00:00,NaN,NaN,NaN,0


In [100]:
import re
from pandas.errors import OutOfBoundsDatetime

df_admin3 = df_admin2.copy()
df_patient1 = df_patient.copy()

def fix_dob_str(s):

    s = str(s).strip()
    if re.match(r'^\d{4}-\d{2}-\d{2}$', s):  
        return s + " 00:00:00"
    return s

df_patient1['dob'] = df_patient1['dob'].astype(str).apply(fix_dob_str)


df_admin3['admittime']  = pd.to_datetime(df_admin3['admittime'],  errors='coerce')
df_admin3['dischtime']  = pd.to_datetime(df_admin3['dischtime'],  errors='coerce')
df_admin3['edregtime']  = pd.to_datetime(df_admin3['edregtime'],  errors='coerce')
df_admin3['edouttime']  = pd.to_datetime(df_admin3['edouttime'],  errors='coerce')
df_patient1['dob']       = pd.to_datetime(df_patient1['dob'],       errors='coerce')

df_merged = df_admin3.merge(
    df_patient1[['subject_id','gender','dob']],
    on='subject_id',
    how='left'
)

df_merged = df_merged.dropna(subset=['admittime', 'dob'])

def safe_diff_days(row):

    try:
        admit_py = row['admittime'].to_pydatetime()
        dob_py   = row['dob'].to_pydatetime()
        diff = admit_py - dob_py
        return diff.days
    except (OverflowError, OutOfBoundsDatetime):
        return np.nan

df_merged['temp_diff'] = df_merged.apply(safe_diff_days, axis=1)

problematic_rows = df_merged[df_merged['temp_diff'].isna()]
if not problematic_rows.empty:
    print("下列行在计算 ADMITTIME - DOB 时发生了溢出或越界：")
    print(problematic_rows[['subject_id','admittime','dob']])

df_merged = df_merged.dropna(subset=['temp_diff'])

df_merged['temp_age'] = df_merged['temp_diff'] / 365.2425
# df_merged = df_merged[(df_merged['temp_age'] >= 0) & (df_merged['temp_age'] < 150)]
df_merged['age'] = df_merged['temp_age'].round(1)
df_merged.drop(['temp_diff','temp_age'], axis=1, inplace=True)

df_merged = df_merged.dropna(subset=['dischtime'])
df_merged['hospital_days'] = (
    (df_merged['dischtime'] - df_merged['admittime']).dt.total_seconds() / (24*3600)
).round(1)

base_cols = ['subject_id','hadm_id','admittime','dischtime','edregtime','edouttime',
             'diagnosis','hospital_expire_flag','gender','age','hospital_days']
df_merged = df_merged[base_cols]

df_chartevent_useful = df_chartevent_filtered2.merge(
    df_ITEMS_chartevent_useful[['itemid','name_full']],
    on='itemid',
    how='left'
)

df_chartevent_merged = df_chartevent_useful.merge(
    df_merged[['subject_id','hadm_id','admittime','dischtime','hospital_expire_flag']],
    on=['subject_id','hadm_id'],
    how='left'
)

mask1 = df_chartevent_merged['hospital_expire_flag'] == 1
df_chartevent_merged_death = df_chartevent_merged[
    (mask1) &
    (df_chartevent_merged['charttime'] >= df_chartevent_merged['dischtime'] - pd.Timedelta(hours=24)) &
    (df_chartevent_merged['charttime'] <= df_chartevent_merged['dischtime'])
]

mask2 = df_chartevent_merged['hospital_expire_flag'] == 0
df_chartevent_merged_alive = df_chartevent_merged[
    (mask2) &
    (df_chartevent_merged['charttime'] >= df_chartevent_merged['admittime']) &
    (df_chartevent_merged['charttime'] <= df_chartevent_merged['dischtime'])
]

df_chartevent_final = pd.concat([df_chartevent_merged_death, df_chartevent_merged_alive], ignore_index=True)

def join_values(series):
    return "~".join(series.astype(str))

df_agg = df_chartevent_final.groupby(
    ['subject_id','hadm_id','name_full'],
    as_index=False
).agg({'valuenum1': join_values})

all_name_full = df_ITEMS_chartevent_useful['name_full'].unique().tolist()

df_pivot = df_agg.pivot_table(
    index=['subject_id','hadm_id'],
    columns='name_full',
    values='valuenum1',
    aggfunc='first'
)

df_pivot = df_pivot.reindex(columns=all_name_full)

df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None

df_result = df_merged.merge(df_pivot, on=['subject_id','hadm_id'], how='left')

df_result

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,fingerstick glucose~1455~nan,ABP [Systolic]~6~nan,Arterial BP [Systolic]~51~nan,Code Status~128~nan,Eye Opening~184~nan,Heart Rate~211~nan,Manual BP [Systolic]~442~nan,Motor Response~454~nan,NBP [Systolic]~455~nan,Resp Rate (Spont)~614~nan,Resp Rate (Total)~615~nan,Respiratory Rate~618~nan,SpO2~646~nan,Spont. Resp. Rate~653~nan,Temperature C~676~nan,Temperature C (calc)~677~nan,Temperature F~678~nan,Temperature F (calc)~679~nan,Verbal Response~723~nan,ALT~769~nan,AST~770~nan,BUN (6-20)~781~nan,Chloride (100-112)~788~nan,Creatinine (0-1.3)~791~nan,Fingerstick Glucose~807~nan,Glucose (70-105)~811~nan,Hemoglobin~814~nan,INR (2-4 ref. range)~815~nan,PTT(22-35)~825~nan,Platelets~828~nan,Potassium (3.5-5.3)~829~nan,albumin~3066~nan,Sodium (135-148)~837~nan,"WBC (4-11,000)~861~nan","WBC (4-11,000)~1127~nan",BUN~1162~nan,FINGERSTICK GLUCOSE.~1310~nan,Bilirubin~4948~nan,bun~5876~nan,WBC 4.0-11.0~4200~nan,Resp. Rate~8113~nan,BUN (6-20)~3737~nan,Blood Glucose~3744~nan,BloodGlucose~3745~nan,Chloride (100-112)~3747~nan,Creatinine (0-0.7)~3750~nan,Platelet (150-440)~3789~nan,Potassium (3.5-5.3)~3792~nan,Ptt~3796~nan,Sodium (135-148)~3803~nan,WhiteBloodC 4.0-11.0~3834~nan,Albumin~1521~nan,Creatinine~1525~nan,Glucose~1529~nan,INR~1530~nan,PTT~1533~nan,Potassium~1535~nan,Sodium~1536~nan,WBC~1542~nan,finger stick glucose~2338~nan,ALBUMIN~2358~nan,Effluent BUN~8220~nan,Platelet Count~6256~nan,ABP [Diastolic]~8364~nan,Arterial BP [Diastolic]~8368~nan,NBP [Diastolic]~8441~nan,BP Cuff [Diastolic]~8502~nan,WBC~220546~nan,ZINR~220561~nan,ZPTT~220562~nan,AST~220587~nan,Chloride (serum)~220602~nan,ZC Reactive Protein (CRP)~220612~nan,Creatinine~220615~nan,ALT~220644~nan,GCS - Eye Opening~220739~nan,Heart Rate~220045~bpm,Arterial Blood Pressure systolic~220050~mmHg,Arterial Blood Pressure diastolic~220051~mmHg,Non Invasive Blood Pressure systolic~220179~mmHg,Non Invasive Blood Pressure diastolic~220180~mmHg,Respiratory Rate~220210~insp/min,Arterial O2 Saturation~220227~%,Hemoglobin~220228~g/dl,O2 saturation pulseoxymetry~220277~%,Code Status~223758~nan,Temperature Fahrenheit~223761~?F,Temperature Celsius~223762~?C,GCS - Verbal Response~223900~nan,GCS - Motor Response~223901~nan,Respiratory Rate (spontaneous)~224689~insp/min,Respiratory Rate (Total)~224690~insp/min,ART BP Systolic~225309~mmHg,ART BP Diastolic~225310~mmHg,BUN~225624~nan,Glucose finger stick~225664~nan,ZPlatelet Count~225678~nan,Total Bilirubin~225690~nan,Sodium (whole blood)~226534~nan,Chloride (whole blood)~226536~nan,Glucose (whole blood)~226537~nan,CreatinineApacheIIScore~226751~nan,CreatinineApacheIIValue~226752~nan,GCSEyeApacheIIValue~226756~nan,GCSMotorApacheIIValue~226757~nan,GCSVerbalApacheIIValue~226758~nan,PotassiumApacheIIScore~226771~nan,PotassiumApacheIIValue~226772~nan,SodiumApacheIIScore~226775~nan,SodiumApacheIIValue~226776~nan,WbcApacheIIScore~226779~nan,WBCApacheIIValue~226780~nan,Bilirubin_ApacheIV~226998~nan,BUN_ApacheIV~227000~nan,BunScore_ApacheIV~227001~nan,Creatinine_ApacheIV~227005~nan,GCSEye_ApacheIV~227011~nan,GCSMotor_ApacheIV~227012~nan,GCSVerbal_ApacheIV~227014~nan,Glucose_ApacheIV~227015~nan,GlucoseScore_ApacheIV~227016~nan,Sodium_ApacheIV~227052~nan,SodiumScore_ApacheIV~227053~nan,TemperatureF_ApacheIV~227054~?F,WBC_ApacheIV~227062~nan,WBCScore_ApacheIV~227063~nan,C Reactive Protein (CRP)~227444~nan,Albumin~227456~nan,Platelet Count~227457~nan,Potassium (whole blood)~227464~nan,PTT~227466~nan,INR~227467~nan,GCSVerbalApacheIIValue (intubated)~228112~nan,Sodium (whole blood) (soft)~228390~nan
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NaT,NEWBORN,0,M,0.0,3.9,NaN,NaN,NaN,NaN,NaN,148.0~131.0~144.0~140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,NaN,302.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [101]:
df_result1 = df_result.copy()

df_result1 = df_result1.dropna(axis=1, how='all')
print(f"drop全为空值的列后shape={df_result1.shape}")

df_result1_over89 = df_result1[df_result1['age']>89]
print(f"超过89岁的老人shape={df_result1_over89.shape}")

df_result1 = df_result1[df_result1['age']>=18]
print(f"drop小于18岁的病人后shape={df_result1.shape}")

df_result1 = df_result1[df_result1['hospital_days']<=60]
print(f"drop住院时长超过60天的病人后shape={df_result1.shape}")

mask = ~(df_result1.iloc[:, 11:].isnull().all(axis=1))
df_result1 = df_result1[mask]
print(f"drop掉input record全为空的病人后shape={df_result1.shape}")

df_result1

drop全为空值的列后shape=(26378, 93)
超过89岁的老人shape=(981, 93)
drop小于18岁的病人后shape=(18477, 93)
drop住院时长超过60天的病人后shape=(18316, 93)
drop掉input record全为空的病人后shape=(17810, 93)


,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,fingerstick glucose~1455~nan,ABP [Systolic]~6~nan,Arterial BP [Systolic]~51~nan,Code Status~128~nan,Eye Opening~184~nan,Heart Rate~211~nan,Manual BP [Systolic]~442~nan,Motor Response~454~nan,NBP [Systolic]~455~nan,Resp Rate (Spont)~614~nan,Resp Rate (Total)~615~nan,Respiratory Rate~618~nan,SpO2~646~nan,Spont. Resp. Rate~653~nan,Temperature C~676~nan,Temperature C (calc)~677~nan,Temperature F~678~nan,Temperature F (calc)~679~nan,Verbal Response~723~nan,ALT~769~nan,AST~770~nan,BUN (6-20)~781~nan,Chloride (100-112)~788~nan,Creatinine (0-1.3)~791~nan,Fingerstick Glucose~807~nan,Glucose (70-105)~811~nan,Hemoglobin~814~nan,INR (2-4 ref. range)~815~nan,PTT(22-35)~825~nan,Platelets~828~nan,Potassium (3.5-5.3)~829~nan,albumin~3066~nan,Sodium (135-148)~837~nan,"WBC (4-11,000)~861~nan","WBC (4-11,000)~1127~nan",BUN~1162~nan,FINGERSTICK GLUCOSE.~1310~nan,Bilirubin~4948~nan,bun~5876~nan,WBC 4.0-11.0~4200~nan,BUN (6-20)~3737~nan,Blood Glucose~3744~nan,BloodGlucose~3745~nan,Chloride (100-112)~3747~nan,Creatinine (0-0.7)~3750~nan,Platelet (150-440)~3789~nan,Potassium (3.5-5.3)~3792~nan,Ptt~3796~nan,Sodium (135-148)~3803~nan,WhiteBloodC 4.0-11.0~3834~nan,Albumin~1521~nan,Creatinine~1525~nan,Glucose~1529~nan,INR~1530~nan,PTT~1533~nan,Potassium~1535~nan,Sodium~1536~nan,WBC~1542~nan,finger stick glucose~2338~nan,ALBUMIN~2358~nan,Platelet Count~6256~nan,ABP [Diastolic]~8364~nan,Arterial BP [Diastolic]~8368~nan,NBP [Diastolic]~8441~nan,BP Cuff [Diastolic]~8502~nan,WBC~220546~nan,AST~220587~nan,Chloride (serum)~220602~nan,Creatinine~220615~nan,ALT~220644~nan,Arterial O2 Saturation~220227~%,Hemoglobin~220228~g/dl,BUN~225624~nan,Total Bilirubin~225690~nan,Sodium (whole blood)~226534~nan,Chloride (whole blood)~226536~nan,Glucose (whole blood)~226537~nan,Albumin~227456~nan,Platelet Count~227457~nan,Potassium (whole blood)~227464~nan,PTT~227466~nan,INR~227467~nan
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,M,76.5,10.8,NaN,NaN,0.0~78.0~73.0~77.0~88.0~73.0~84.0~89.0~100.0~1...,Full Code~Do Not Resuscita~Full Code~Full Code...,1.0~1.0~3.0~1.0~1.0~4.0~3.0~3.0~3.0~3.0~4.0~3....,151.0~135.0~143.0~165.0~168.0~147.0~135.0~144....,NaN,1.0~1.0~nan~4.0~4.0~6.0~6.0~6.0~6.0~6.0~6.0~6....,217.0~102.0~94.0~80.0~91.0~86.0~96.0~64.0~83.0...,8.0~3.0~0.0~0.0~0.0~0.0~0.0~0.0~0.0~0.0~0.0~4....,24.0~19.0~18.0~18.0~18.0~18.0~18.0~18.0~16.0~1...,5.0~21.0~21.0~15.0~0.0~0.0~21.0~0.0~0.0~5.0~16...,74.0~99.0~96.0~98.0~100.0~100.0~100.0~100.0~99...,NaN,36.20000076293945~36.20000076293945~36.5~36.59...,36.77780151367188~36.33330154418945~36.1666984...,98.1999969482422~97.4000015258789~97.099998474...,97.16000366210938~97.16000366210938~97.6999969...,1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1....,25.0~20.0,69.0~67.0,41.0~43.0~42.0~36.0~32.0~30.0~24.0~20.0~17.0,111.0~111.0~110.0~111.0~112.0~111.0~110.0~109....,2.4~2.5~2.5~1.9~1.7~1.6~1.5~1.3~1.3,306.0~189.0~136.0~120.0~126.0~105.0~100.0~102....,140.0~162.0~281.0~265.0~267.0~299.0~294.0~293....,7.800000190734863~8.199999809265137~9.69999980...,1.7~1.7~1.5~1.5~1.5~1.6,58.3~46.4~36.2~150.0~150.0~125.7~150.0~110.2~9...,190.0~253.0~359.0~219.0~179.0~133.0~121.0~155....,4.4~4.0~4.0~4.1~3.7~3.9~4.4~3.9~4.9~4.2~4.3~4....,NaN,138.0~153.0~143.0~140.0~139.0~138.0~139.0~138....,11.3~19.1~24.4~18.8~15.1~12.5~10.3~10.7~10.5,11.3~19.1~24.4~18.8~15.1~12.5~10.3~10.7~10.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0~49.0~47.0~51.0~54.0~43.0~53.0~57.0~60.0~66...,38.0~64.0~41.0~46.0~63.0~36.0~31.0~28.0~38.0~3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,M,41.8,4.9,NaN,NaN,155.0~153.0~168.0~181.0~145.0~98.0~67.0~83.0~8...,Full Code~Full Code~Full Code~Full Code~Full C...,1.0~1.

In [102]:
filename = 'NOTEEVENTS.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df_note = pd.read_csv(file)
df_note

C:\Users\AaronChen\AppData\Local\Temp\ipykernel_16908\228834435.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_note = pd.read_csv(file)


,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,738407,20409,NaN,2119-01-04 00:00:00,2119-01-04 12:59:00,NaN,Radiology,ABDOMEN U.S. (COMPLETE STUDY),NaN,NaN,[**2119-1-4**] 12:59 PM\n ABDOMEN U.S. (COMPLE...
1,738408,20409,NaN,2119-01-09 00:00:00,2119-01-09 13:05:00,NaN,Radiology,MR LIVER WITH CONTRAST,NaN,NaN,[**2119-1-9**] 1:05 PM\n MR LIVER WITH CONTRAS...
2,738409,20409,NaN,2119-01-16 00:00:00,2119-01-16 21:24:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2119-1-16**] 9:24 PM\n CHEST (PORTABLE AP) ...
3,738410,20409,NaN,2119-01-18 00:00:00,2119-01-18 13:24:00,NaN,Radiology,CT ABD W&W/O C,NaN,NaN,[**2119-1-18**] 1:24 PM\n CT ABD W&W/O C; CT P...
4,738411,20409,NaN,2119-01-18 00:00:00,2119-01-18 15:45:00,NaN,Radiology,PARACENTESIS DIAG. OR THERAPEUTIC,NaN,NaN,[**2119-1-18**] 3:45 PM\n PARACENTESIS DIAG. O...
...,...,...,...,...,...,...,...,...,...,...,...
880102,2083175,32806,104049.0,2162-11-02 00:00:00,2162-11-02 13:09:00,2162-11-02 13:27:00,Nursing/other,Report,14461.0,NaN,1. FEN\nTF= min140cc/k/d of BM/E24. min47cc q4...
880103,2083176,32806,104049.0,2162-11-02 00:00:00,2162-11-02 13:09:00,2162-11-02 16:36:00,Nursing/other,Report,16865.0,NaN,I have examined pt. & agree w/ [**First Name8 ...
880104,2083177,32806,104049.0,2162-11-03 00:00:00,2162-11-03 01:57:00,2162-11-03 02:13:00,Nursing/other,Report,17604.0,NaN,NPN nights\n\n\nFluids/Nutrition: Weight 2025...
880105,2083178,32806,104049.0,2162-11-03 00:00:00,2162-11-03 09:55:00,2162-11-03 09:59:00,Nursing/other,Report,21222.0,NaN,Attending Note\nDay of life 12 PMA 35 [**2-7**...


In [103]:
df_note1 = df_note.copy()
df_note1 = df_note1[~df_note1['hadm_id'].isna()]
df_note1 = df_note1[['subject_id','hadm_id','text']]
df_note1

,subject_id,hadm_id,text
12,20409,102314.0,"[**2119-2-3**] 4:10 PM\n US ABD LIMIT, SINGLE ..."
13,20409,102314.0,"[**2119-2-4**] 10:38 AM\n US ABD LIMIT, SINGLE..."
14,25080,185945.0,[**2167-2-14**] 10:41 AM\n CHEST (PA & LAT) ...
15,25080,185945.0,[**2167-2-16**] 7:50 PM\n CHEST (PORTABLE AP) ...
16,25080,185945.0,[**2167-2-17**] 2:24 AM\n CHEST (PORTABLE AP) ...
...,...,...,...
880102,32806,104049.0,1. FEN\nTF= min140cc/k/d of BM/E24. min47cc q4...
880103,32806,104049.0,I have examined pt. & agree w/ [**First Name8 ...
880104,32806,104049.0,NPN nights\n\n\nFluids/Nutrition: Weight 2025...
880105,32806,104049.0,Attending Note\nDay of life 12 PMA 35 [**2-7**...


In [104]:
subject_counts_per_hadm = df_note1.groupby('hadm_id')['subject_id'].nunique()

hadm_ids_with_multiple_subjects = subject_counts_per_hadm[subject_counts_per_hadm > 1].index

filtered_df = df_note1[df_note1['hadm_id'].isin(hadm_ids_with_multiple_subjects)]
filtered_df

,subject_id,hadm_id,text


In [105]:
df_note2 = df_note1.copy()
df_note2 = df_note2.groupby(['subject_id', 'hadm_id'])['text'].agg('; '.join).reset_index()
df_note2 = df_note2[df_note2['hadm_id'].isin(df_result1['hadm_id'])]
df_note2

,subject_id,hadm_id,text
1,3,145834.0,[**2101-10-20**] 5:49 PM\n CT ABDOMEN W/O CONT...
5,9,150750.0,[**2149-11-9**] 11:21 AM\n CT HEAD W/O CONTRAS...
7,11,194540.0,[**2178-4-16**] 5:01 AM\n MR HEAD W & W/O CONT...
9,13,143045.0,[**2167-1-9**] 6:11 AM\n CHEST (PORTABLE AP) ...
11,19,109235.0,[**2108-8-5**] 8:13 AM\n TRAUMA #2 (AP CXR & P...
...,...,...,...
26652,32781,142684.0,[**2134-9-14**] 11:52 PM\n CHEST (PORTABLE AP)...
26656,32786,163011.0,[**2152-12-9**] 4:00 PM\n CHEST (PORTABLE AP) ...
26657,32786,165351.0,[**2152-11-15**] 6:52 PM\n CHEST (PORTABLE AP)...
26658,32787,157308.0,[**2102-10-2**] 2:46 PM\n CHEST PORT. LINE PLA...


In [107]:
import re
df_note3 = df_note2.copy()

pattern = r'''(?i)
(
    do\s*not\s*resuscitate          # do not resuscitate
    | dnr                           
    | do\s*not\s*attempt\s*resuscitation   
    | refuse(d)?\s+(life[-\s]*sustaining\s*treatment|life\s*support)
    | withhold\s+(life[-\s]*sustaining\s*treatment|life\s*support)
    | withdraw\s+(life[-\s]*sustaining\s*treatment|life\s*support)
)
'''

mask_exclude = df_note3['text'].str.contains(pattern, na=False)

num_excluded = mask_exclude.sum()
print(f"匹配到需排除的记录数：{num_excluded}")

# 4) 若要查看哪些行被排除，可执行：
df_excluded = df_note3[mask_exclude]
# print(df_excluded)

# 5) 最终保留下不包含这些词的记录
df_filtered = df_note3[~mask_exclude]

# 6) 现在 df_filtered 就是排除了“拒绝生命维持治疗”或具有“do not resuscitate”指令记录后的结果
print("排除后的数据行数:", len(df_filtered))
df_filtered

匹配到需排除的记录数：0
排除后的数据行数: 17748


C:\Users\AaronChen\AppData\Local\Temp\ipykernel_16908\2996590125.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_exclude = df_note3['text'].str.contains(pattern, na=False)


,subject_id,hadm_id,text
1,3,145834.0,[**2101-10-20**] 5:49 PM\n CT ABDOMEN W/O CONT...
5,9,150750.0,[**2149-11-9**] 11:21 AM\n CT HEAD W/O CONTRAS...
7,11,194540.0,[**2178-4-16**] 5:01 AM\n MR HEAD W & W/O CONT...
9,13,143045.0,[**2167-1-9**] 6:11 AM\n CHEST (PORTABLE AP) ...
11,19,109235.0,[**2108-8-5**] 8:13 AM\n TRAUMA #2 (AP CXR & P...
...,...,...,...
26652,32781,142684.0,[**2134-9-14**] 11:52 PM\n CHEST (PORTABLE AP)...
26656,32786,163011.0,[**2152-12-9**] 4:00 PM\n CHEST (PORTABLE AP) ...
26657,32786,165351.0,[**2152-11-15**] 6:52 PM\n CHEST (PORTABLE AP)...
26658,32787,157308.0,[**2102-10-2**] 2:46 PM\n CHEST PORT. LINE PLA...


In [108]:
filename = '/mimic-iii-clinical-database-carevue-subset-1.4/DRGCODES.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df_DRG = pd.read_csv(file)
df_DRG

,row_id,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
0,2,25080,185945,HCFA,104,CARDIAC VALVE & OTHER MAJOR CARDIOTHORACIC PRO...,NaN,NaN
1,3,11677,137776,HCFA,202,CIRRHOSIS & ALCOHOLIC HEPATITIS,NaN,NaN
2,4,20409,102314,HCFA,483,"TRACHEOSTOMY EXCEPT FOR FACE, MOUTH, & NECK DI...",NaN,NaN
3,5,11705,165730,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
4,6,16178,138464,HCFA,109,CORONARY BYPASS WITHOUT CARDIAC CATHETER,NaN,NaN
...,...,...,...,...,...,...,...,...
38914,57725,32433,156768,APR,9523,Nonextensive Procedure Unrelated To Principal ...,3.0,2.0
38915,57726,32433,156768,APR,9523,Nonextensive Procedure Unrelated To Principal ...,3.0,2.0
38916,58031,31145,115401,APR,1152,"Other Ear, Nose, Mouth,Throat & Cranial/Facial...",2.0,1.0
38917,58032,31145,115401,APR,1152,"Other Ear, Nose, Mouth,Throat & Cranial/Facial...",2.0,1.0


In [112]:
df_result2 = df_result1.copy()

def unique_entries(cell):
    if pd.isna(cell):
        return cell
    items = cell.split('~')
    unique_items = set(items)
    return '~'.join(unique_items)

df_result2['Code Status~128~nan'] = df_result2['Code Status~128~nan'].apply(unique_entries)
df_result2

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,fingerstick glucose~1455~nan,ABP [Systolic]~6~nan,Arterial BP [Systolic]~51~nan,Code Status~128~nan,Eye Opening~184~nan,Heart Rate~211~nan,Manual BP [Systolic]~442~nan,Motor Response~454~nan,NBP [Systolic]~455~nan,Resp Rate (Spont)~614~nan,Resp Rate (Total)~615~nan,Respiratory Rate~618~nan,SpO2~646~nan,Spont. Resp. Rate~653~nan,Temperature C~676~nan,Temperature C (calc)~677~nan,Temperature F~678~nan,Temperature F (calc)~679~nan,Verbal Response~723~nan,ALT~769~nan,AST~770~nan,BUN (6-20)~781~nan,Chloride (100-112)~788~nan,Creatinine (0-1.3)~791~nan,Fingerstick Glucose~807~nan,Glucose (70-105)~811~nan,Hemoglobin~814~nan,INR (2-4 ref. range)~815~nan,PTT(22-35)~825~nan,Platelets~828~nan,Potassium (3.5-5.3)~829~nan,albumin~3066~nan,Sodium (135-148)~837~nan,"WBC (4-11,000)~861~nan","WBC (4-11,000)~1127~nan",BUN~1162~nan,FINGERSTICK GLUCOSE.~1310~nan,Bilirubin~4948~nan,bun~5876~nan,WBC 4.0-11.0~4200~nan,BUN (6-20)~3737~nan,Blood Glucose~3744~nan,BloodGlucose~3745~nan,Chloride (100-112)~3747~nan,Creatinine (0-0.7)~3750~nan,Platelet (150-440)~3789~nan,Potassium (3.5-5.3)~3792~nan,Ptt~3796~nan,Sodium (135-148)~3803~nan,WhiteBloodC 4.0-11.0~3834~nan,Albumin~1521~nan,Creatinine~1525~nan,Glucose~1529~nan,INR~1530~nan,PTT~1533~nan,Potassium~1535~nan,Sodium~1536~nan,WBC~1542~nan,finger stick glucose~2338~nan,ALBUMIN~2358~nan,Platelet Count~6256~nan,ABP [Diastolic]~8364~nan,Arterial BP [Diastolic]~8368~nan,NBP [Diastolic]~8441~nan,BP Cuff [Diastolic]~8502~nan,WBC~220546~nan,AST~220587~nan,Chloride (serum)~220602~nan,Creatinine~220615~nan,ALT~220644~nan,Arterial O2 Saturation~220227~%,Hemoglobin~220228~g/dl,BUN~225624~nan,Total Bilirubin~225690~nan,Sodium (whole blood)~226534~nan,Chloride (whole blood)~226536~nan,Glucose (whole blood)~226537~nan,Albumin~227456~nan,Platelet Count~227457~nan,Potassium (whole blood)~227464~nan,PTT~227466~nan,INR~227467~nan
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,M,76.5,10.8,NaN,NaN,0.0~78.0~73.0~77.0~88.0~73.0~84.0~89.0~100.0~1...,Full Code~Do Not Resuscita,1.0~1.0~3.0~1.0~1.0~4.0~3.0~3.0~3.0~3.0~4.0~3....,151.0~135.0~143.0~165.0~168.0~147.0~135.0~144....,NaN,1.0~1.0~nan~4.0~4.0~6.0~6.0~6.0~6.0~6.0~6.0~6....,217.0~102.0~94.0~80.0~91.0~86.0~96.0~64.0~83.0...,8.0~3.0~0.0~0.0~0.0~0.0~0.0~0.0~0.0~0.0~0.0~4....,24.0~19.0~18.0~18.0~18.0~18.0~18.0~18.0~16.0~1...,5.0~21.0~21.0~15.0~0.0~0.0~21.0~0.0~0.0~5.0~16...,74.0~99.0~96.0~98.0~100.0~100.0~100.0~100.0~99...,NaN,36.20000076293945~36.20000076293945~36.5~36.59...,36.77780151367188~36.33330154418945~36.1666984...,98.1999969482422~97.4000015258789~97.099998474...,97.16000366210938~97.16000366210938~97.6999969...,1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1....,25.0~20.0,69.0~67.0,41.0~43.0~42.0~36.0~32.0~30.0~24.0~20.0~17.0,111.0~111.0~110.0~111.0~112.0~111.0~110.0~109....,2.4~2.5~2.5~1.9~1.7~1.6~1.5~1.3~1.3,306.0~189.0~136.0~120.0~126.0~105.0~100.0~102....,140.0~162.0~281.0~265.0~267.0~299.0~294.0~293....,7.800000190734863~8.199999809265137~9.69999980...,1.7~1.7~1.5~1.5~1.5~1.6,58.3~46.4~36.2~150.0~150.0~125.7~150.0~110.2~9...,190.0~253.0~359.0~219.0~179.0~133.0~121.0~155....,4.4~4.0~4.0~4.1~3.7~3.9~4.4~3.9~4.9~4.2~4.3~4....,NaN,138.0~153.0~143.0~140.0~139.0~138.0~139.0~138....,11.3~19.1~24.4~18.8~15.1~12.5~10.3~10.7~10.5,11.3~19.1~24.4~18.8~15.1~12.5~10.3~10.7~10.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0~49.0~47.0~51.0~54.0~43.0~53.0~57.0~60.0~66...,38.0~64.0~41.0~46.0~63.0~36.0~31.0~28.0~38.0~3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,M,41.8,4.9,NaN,NaN,155.0~153.0~168.0~181.0~145.0~98.0~67.0~83.0~8...,Full Code,1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,82.0~85.0~83.0~83.0~9

In [113]:
df_result2['Code Status~128~nan'] = df_result2['Code Status~128~nan'].replace('^\s*nan\s*$', np.nan, regex=True)

In [114]:
df_result2['Code Status~128~nan'].value_counts().index

Index(['Full Code', 'Do Not Resuscita', 'Full Code~Do Not Resuscita',
       'Comfort Measures~Do Not Resuscita', 'Comfort Measures~Full Code',
       'Comfort Measures', 'Comfort Measures~Full Code~Do Not Resuscita',
       'nan~Full Code', 'Full Code~CPR Not Indicate', 'Do Not Intubate',
       'CPR Not Indicate', 'Full Code~Do Not Intubate',
       'nan~Full Code~Do Not Resuscita', 'Full Code~Other/Remarks',
       'Comfort Measures~Full Code~CPR Not Indicate',
       'Do Not Resuscita~Do Not Intubate',
       'Do Not Resuscita~Full Code~Do Not Intubate',
       'Do Not Resuscita~CPR Not Indicate',
       'Comfort Measures~CPR Not Indicate',
       'Do Not Intubate~Full Code~Do Not Resuscita',
       'Do Not Intubate~Do Not Resuscita', 'Other/Remarks',
       'Full Code~Do Not Resuscita~CPR Not Indicate', 'nan~Do Not Resuscita',
       'Comfort Measures~Do Not Intubate',
       'Full Code~Other/Remarks~Do Not Resuscita',
       'Do Not Resuscita~Full Code~CPR Not Indicate',
       '

In [115]:
df_result2['Code Status~128~nan'].value_counts()

Code Status~128~nan
Full Code                                                             13620
Do Not Resuscita                                                       1116
Full Code~Do Not Resuscita                                              945
Comfort Measures~Do Not Resuscita                                       386
Comfort Measures~Full Code                                              255
Comfort Measures                                                        180
Comfort Measures~Full Code~Do Not Resuscita                             146
nan~Full Code                                                           100
Full Code~CPR Not Indicate                                               37
Do Not Intubate                                                          33
CPR Not Indicate                                                         32
Full Code~Do Not Intubate                                                29
nan~Full Code~Do Not Resuscita                                      

In [116]:
df_result3 = df_result2.copy()

# Normalize text to ensure uniformity in checks
df_result3['Code Status~128~nan'] = df_result3['Code Status~128~nan'].str.lower()

# Define allowed values list
allowed_values_128 = ['full code', 'full code~nan', 'other/remarks~full code', 'other/remarks', 'full code~other/remarks']

# Apply filters with improved readability and efficiency
df_result3 = df_result3[
    (
        (df_result3['Code Status~128~nan'].isna() | df_result3['Code Status~128~nan'].isin(allowed_values_128))
    )
]
df_result3

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,fingerstick glucose~1455~nan,ABP [Systolic]~6~nan,Arterial BP [Systolic]~51~nan,Code Status~128~nan,Eye Opening~184~nan,Heart Rate~211~nan,Manual BP [Systolic]~442~nan,Motor Response~454~nan,NBP [Systolic]~455~nan,Resp Rate (Spont)~614~nan,Resp Rate (Total)~615~nan,Respiratory Rate~618~nan,SpO2~646~nan,Spont. Resp. Rate~653~nan,Temperature C~676~nan,Temperature C (calc)~677~nan,Temperature F~678~nan,Temperature F (calc)~679~nan,Verbal Response~723~nan,ALT~769~nan,AST~770~nan,BUN (6-20)~781~nan,Chloride (100-112)~788~nan,Creatinine (0-1.3)~791~nan,Fingerstick Glucose~807~nan,Glucose (70-105)~811~nan,Hemoglobin~814~nan,INR (2-4 ref. range)~815~nan,PTT(22-35)~825~nan,Platelets~828~nan,Potassium (3.5-5.3)~829~nan,albumin~3066~nan,Sodium (135-148)~837~nan,"WBC (4-11,000)~861~nan","WBC (4-11,000)~1127~nan",BUN~1162~nan,FINGERSTICK GLUCOSE.~1310~nan,Bilirubin~4948~nan,bun~5876~nan,WBC 4.0-11.0~4200~nan,BUN (6-20)~3737~nan,Blood Glucose~3744~nan,BloodGlucose~3745~nan,Chloride (100-112)~3747~nan,Creatinine (0-0.7)~3750~nan,Platelet (150-440)~3789~nan,Potassium (3.5-5.3)~3792~nan,Ptt~3796~nan,Sodium (135-148)~3803~nan,WhiteBloodC 4.0-11.0~3834~nan,Albumin~1521~nan,Creatinine~1525~nan,Glucose~1529~nan,INR~1530~nan,PTT~1533~nan,Potassium~1535~nan,Sodium~1536~nan,WBC~1542~nan,finger stick glucose~2338~nan,ALBUMIN~2358~nan,Platelet Count~6256~nan,ABP [Diastolic]~8364~nan,Arterial BP [Diastolic]~8368~nan,NBP [Diastolic]~8441~nan,BP Cuff [Diastolic]~8502~nan,WBC~220546~nan,AST~220587~nan,Chloride (serum)~220602~nan,Creatinine~220615~nan,ALT~220644~nan,Arterial O2 Saturation~220227~%,Hemoglobin~220228~g/dl,BUN~225624~nan,Total Bilirubin~225690~nan,Sodium (whole blood)~226534~nan,Chloride (whole blood)~226536~nan,Glucose (whole blood)~226537~nan,Albumin~227456~nan,Platelet Count~227457~nan,Potassium (whole blood)~227464~nan,PTT~227466~nan,INR~227467~nan
5,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,M,41.8,4.9,NaN,NaN,155.0~153.0~168.0~181.0~145.0~98.0~67.0~83.0~8...,full code,1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,82.0~85.0~83.0~83.0~92.0~96.0~93.0~93.0~89.0~8...,NaN,1.0~2.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,NaN,5.0~0.0~0.0~3.0~0.0~0.0~0.0,19.0~20.0~20.0~23.0~18.0~14.0~14.0,17.0~19.0~17.0~19.0~18.0~14.0~14.0~14.0~14.0~1...,95.0~96.0~96.0~96.0~100.0~95.0~93.0~95.0~96.0~...,NaN,NaN,36.77780151367188~36.83330154418945~36.1110992...,98.1999969482422~98.3000030517578~97.0~96.0~94...,NaN,1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,NaN,NaN,28.0~33.0,98.0~107.0,1.7~2.0,212.0~146.0~152.0~116.0,184.0~114.0,11.5~12.600000381469728,1.1,23.3,306.0~330.0,3.7~3.5,NaN,129.0~131.0~133.0~137.0~137.0,12.2~11.1,12.2~11.1,28.0~33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7~2.0,184.0~114.0,1.1,23.3,3.7~3.5,129.0~131.0~133.0~137.0~137.0,12.2~11.1,NaN,NaN,NaN,NaN,78.0~78.0~81.0~85.0~67.0~59.0~47.0~52.0~49.0~5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,F,50.1,25.5,NaN,NaN,NaN,full code,4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4....,86.0~90.0~95.0~97.0~89.0~93.0~80.0~90.0~89.0~8...,NaN,6.0~6.0~6.0~6.0~6.0~6.0~6.0~6.0~5.0~6.0~5.0~6....,115.0~112.0~106.0~104.0~100.0~98.0~115.0~107.0...,NaN,NaN,17.0~20.0~18.0~20.0~15.0~19.0~17.0~18.0~18.0~2...,96.0~97.0~96.0~96.0~96.0~97.0~93.0~99.0~99.0~9...,NaN,NaN,36.88890075683594~36.83330154418945~37.3889007...,98.4000015258789~98.3000030517578~99.300003051...,NaN,5.0~5.0~5.0~5.0~5.0~5.0~5.0~5.0~5.0~5.0~4.0~5....,NaN,NaN,14.0~19.0,106.0~106.0,0.7~0.7,120.0~149.0~119.0~116.0,129.0~130.0~126.0,12.600000381469728~12.5,NaN,NaN,258.0~243.0,3.9~3.8,NaN,141.0~142.0,9.3~12.8,9.3~12.8,14.0~19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7~0.7,129.0~130.0~126.0,NaN,NaN

In [117]:
df_result3['Code Status~128~nan'].value_counts()

Code Status~128~nan
full code                  13620
full code~other/remarks       16
other/remarks                  8
Name: count, dtype: int64

In [123]:
df_result4 = df_result3.dropna(thresh=df_result3.shape[0]*0.01, axis=1)
df_result4

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,Arterial BP [Systolic]~51~nan,Code Status~128~nan,Eye Opening~184~nan,Heart Rate~211~nan,Manual BP [Systolic]~442~nan,Motor Response~454~nan,NBP [Systolic]~455~nan,Resp Rate (Spont)~614~nan,Resp Rate (Total)~615~nan,Respiratory Rate~618~nan,SpO2~646~nan,Temperature C~676~nan,Temperature C (calc)~677~nan,Temperature F~678~nan,Temperature F (calc)~679~nan,Verbal Response~723~nan,ALT~769~nan,AST~770~nan,BUN (6-20)~781~nan,Chloride (100-112)~788~nan,Creatinine (0-1.3)~791~nan,Fingerstick Glucose~807~nan,Glucose (70-105)~811~nan,Hemoglobin~814~nan,INR (2-4 ref. range)~815~nan,PTT(22-35)~825~nan,Platelets~828~nan,Potassium (3.5-5.3)~829~nan,Sodium (135-148)~837~nan,"WBC (4-11,000)~861~nan","WBC (4-11,000)~1127~nan",BUN~1162~nan,Albumin~1521~nan,Creatinine~1525~nan,Glucose~1529~nan,INR~1530~nan,PTT~1533~nan,Potassium~1535~nan,Sodium~1536~nan,WBC~1542~nan,Arterial BP [Diastolic]~8368~nan,NBP [Diastolic]~8441~nan
5,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,M,41.8,4.9,155.0~153.0~168.0~181.0~145.0~98.0~67.0~83.0~8...,full code,1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,82.0~85.0~83.0~83.0~92.0~96.0~93.0~93.0~89.0~8...,NaN,1.0~2.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,NaN,5.0~0.0~0.0~3.0~0.0~0.0~0.0,19.0~20.0~20.0~23.0~18.0~14.0~14.0,17.0~19.0~17.0~19.0~18.0~14.0~14.0~14.0~14.0~1...,95.0~96.0~96.0~96.0~100.0~95.0~93.0~95.0~96.0~...,NaN,36.77780151367188~36.83330154418945~36.1110992...,98.1999969482422~98.3000030517578~97.0~96.0~94...,NaN,1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,NaN,NaN,28.0~33.0,98.0~107.0,1.7~2.0,212.0~146.0~152.0~116.0,184.0~114.0,11.5~12.600000381469728,1.1,23.3,306.0~330.0,3.7~3.5,129.0~131.0~133.0~137.0~137.0,12.2~11.1,12.2~11.1,28.0~33.0,NaN,1.7~2.0,184.0~114.0,1.1,23.3,3.7~3.5,129.0~131.0~133.0~137.0~137.0,12.2~11.1,78.0~78.0~81.0~85.0~67.0~59.0~47.0~52.0~49.0~5...,NaN
7,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,F,50.1,25.5,NaN,full code,4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4....,86.0~90.0~95.0~97.0~89.0~93.0~80.0~90.0~89.0~8...,NaN,6.0~6.0~6.0~6.0~6.0~6.0~6.0~6.0~5.0~6.0~5.0~6....,115.0~112.0~106.0~104.0~100.0~98.0~115.0~107.0...,NaN,NaN,17.0~20.0~18.0~20.0~15.0~19.0~17.0~18.0~18.0~2...,96.0~97.0~96.0~96.0~96.0~97.0~93.0~99.0~99.0~9...,NaN,36.88890075683594~36.83330154418945~37.3889007...,98.4000015258789~98.3000030517578~99.300003051...,NaN,5.0~5.0~5.0~5.0~5.0~5.0~5.0~5.0~5.0~5.0~4.0~5....,NaN,NaN,14.0~19.0,106.0~106.0,0.7~0.7,120.0~149.0~119.0~116.0,129.0~130.0~126.0,12.600000381469728~12.5,NaN,NaN,258.0~243.0,3.9~3.8,141.0~142.0,9.3~12.8,9.3~12.8,14.0~19.0,NaN,0.7~0.7,129.0~130.0~126.0,NaN,NaN,3.9~3.8,141.0~142.0,9.3~12.8,NaN,58.0~56.0~50.0~42.0~52.0~47.0~61.0~57.0~64.0~7...
9,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaT,NaT,CORONARY ARTERY DISEASE,0,F,39.9,6.9,151.0~147.0~151.0~135.0~127.0~121.0~105.0~114....,full code,4.0~4.0~4.0~4.0~4.0~1.0~3.0~3.0~4.0~4.0~4.0~4....,80.0~77.0~78.0~90.0~92.0~76.0~66.0~60.0~64.0~6...,NaN,6.0~6.0~6.0~6.0~6.0~1.0~4.0~4.0~6.0~6.0~6.0~6....,112.0~98.0~117.0~102.0,0.0~nan,14.0~28.0~26.0~nan,19.0~15.0~20.0~16.0~19.0~18.0~15.0~12.0~14.0~1...,98.0~98.0~96.0~95.0~95.0~94.0~95.0~96.0~96.0~9...,36.5~36.59999847412109~36.29999923706055~36.59...,37.27780151367188~36.44440078735352~35.9444007...,99.0999984741211~97.5999984741211~96.699996948...,97.6999969482422~97.87999725341795~97.33999633...,5.0~5.0~5.0~5.0~5.0~1.0~1.0~1.0~1.0~5.0~5.0~5....,16.0,16.0,13.0~18.0~15.0~14.0~16.0~13.0,106.0~108.0~110.0~108.0~104.0,0.6~0.8~0.5~0.6~0.6~0.4,149.0~205.0~109.0~197.0~164.0~137.0,169.0~164.0~197.0~135.0~239.0~154.0~119.0~158....,12.300000190734863~12.0~10.800000190734863~10....,1.2~1.2~1.8~1.4,44.1~32.7~30.8~27.9,216.0~213.0~115.0~92.0~74.0~103.0,3.5~4.3~3.9~5.3~5.0~3.7~3.0~4.3~3.9~4.7~3.7~4....,139.0~138.0~137.0~137.0~140.0~140.0~1

In [124]:
df_result4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14404 entries, 5 to 26377
Data columns (total 53 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   subject_id                        14404 non-null  int64         
 1   hadm_id                           14404 non-null  int64         
 2   admittime                         14404 non-null  datetime64[ns]
 3   dischtime                         14404 non-null  datetime64[ns]
 4   edregtime                         7279 non-null   datetime64[ns]
 5   edouttime                         7279 non-null   datetime64[ns]
 6   diagnosis                         14404 non-null  object        
 7   hospital_expire_flag              14404 non-null  int64         
 8   gender                            14404 non-null  object        
 9   age                               14404 non-null  float64       
 10  hospital_days                     14404 non-null  f

In [125]:
col_head = df_result4.columns.tolist()[:11]
col_tail = df_result4.columns.tolist()[11:]
col_tail_new = []
for each1 in useful_cols:
    for each2 in col_tail:
        temp = each2.split('~')[0]
        if each1 == temp:
            col_tail_new.append(each2)
col_new = col_head + col_tail_new
col_new

['subject_id',
 'hadm_id',
 'admittime',
 'dischtime',
 'edregtime',
 'edouttime',
 'diagnosis',
 'hospital_expire_flag',
 'gender',
 'age',
 'hospital_days',
 'Resp Rate (Spont)~614~nan',
 'Resp Rate (Total)~615~nan',
 'Respiratory Rate~618~nan',
 'Heart Rate~211~nan',
 'Arterial BP [Systolic]~51~nan',
 'Manual BP [Systolic]~442~nan',
 'NBP [Systolic]~455~nan',
 'Arterial BP [Diastolic]~8368~nan',
 'NBP [Diastolic]~8441~nan',
 'Temperature C~676~nan',
 'Temperature C (calc)~677~nan',
 'Temperature F~678~nan',
 'Temperature F (calc)~679~nan',
 'SpO2~646~nan',
 'Eye Opening~184~nan',
 'Verbal Response~723~nan',
 'Motor Response~454~nan',
 'ALT~769~nan',
 'AST~770~nan',
 'Albumin~1521~nan',
 'BUN~1162~nan',
 'BUN (6-20)~781~nan',
 'Chloride (100-112)~788~nan',
 'Creatinine (0-1.3)~791~nan',
 'Creatinine~1525~nan',
 'Fingerstick Glucose~807~nan',
 'Glucose (70-105)~811~nan',
 'Glucose~1529~nan',
 'Hemoglobin~814~nan',
 'INR (2-4 ref. range)~815~nan',
 'INR~1530~nan',
 'Platelets~828~nan',

In [140]:
df_result5 = df_result4.copy()
df_result5 = df_result5[col_new]

list1 = ['Temperature F~678~nan', 'Temperature F (calc)~679~nan']
def trans(values):
    if pd.isna(values):  # Correct way to check for NaN
        return values
    else:
        m = str(values).split('~')
        n = []
        for each in m:
            try:
                if each != 'nan' and each.strip():  # Check if not 'nan' and not empty
                    f_value = float(each)
                    c_value = (f_value - 32) * 5 / 9  # Convert F to C
                    n.append(f"{c_value:.2f}")  # Format the result to 2 decimal places
            except ValueError:
                n.append('error')  # Handle possible conversion errors

        return "~".join(n)  # Join all converted values back with '~'

# Applying the transformation function to each specified column
for each3 in list1:
    df_result5[each3] = df_result5[each3].apply(trans)

df_result5

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,Resp Rate (Spont)~614~nan,Resp Rate (Total)~615~nan,Respiratory Rate~618~nan,Heart Rate~211~nan,Arterial BP [Systolic]~51~nan,Manual BP [Systolic]~442~nan,NBP [Systolic]~455~nan,Arterial BP [Diastolic]~8368~nan,NBP [Diastolic]~8441~nan,Temperature C~676~nan,Temperature C (calc)~677~nan,Temperature F~678~nan,Temperature F (calc)~679~nan,SpO2~646~nan,Eye Opening~184~nan,Verbal Response~723~nan,Motor Response~454~nan,ALT~769~nan,AST~770~nan,Albumin~1521~nan,BUN~1162~nan,BUN (6-20)~781~nan,Chloride (100-112)~788~nan,Creatinine (0-1.3)~791~nan,Creatinine~1525~nan,Fingerstick Glucose~807~nan,Glucose (70-105)~811~nan,Glucose~1529~nan,Hemoglobin~814~nan,INR (2-4 ref. range)~815~nan,INR~1530~nan,Platelets~828~nan,Potassium (3.5-5.3)~829~nan,Potassium~1535~nan,Sodium (135-148)~837~nan,Sodium~1536~nan,"WBC (4-11,000)~1127~nan","WBC (4-11,000)~861~nan",WBC~1542~nan,PTT(22-35)~825~nan,PTT~1533~nan,Code Status~128~nan
5,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,M,41.8,4.9,5.0~0.0~0.0~3.0~0.0~0.0~0.0,19.0~20.0~20.0~23.0~18.0~14.0~14.0,17.0~19.0~17.0~19.0~18.0~14.0~14.0~14.0~14.0~1...,82.0~85.0~83.0~83.0~92.0~96.0~93.0~93.0~89.0~8...,155.0~153.0~168.0~181.0~145.0~98.0~67.0~83.0~8...,NaN,NaN,78.0~78.0~81.0~85.0~67.0~59.0~47.0~52.0~49.0~5...,NaN,NaN,36.77780151367188~36.83330154418945~36.1110992...,36.78~36.83~36.11~35.56~34.89~35.56~35.78~37.0...,NaN,95.0~96.0~96.0~96.0~100.0~95.0~93.0~95.0~96.0~...,1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,1.0~2.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0~1.0,NaN,NaN,NaN,28.0~33.0,28.0~33.0,98.0~107.0,1.7~2.0,1.7~2.0,212.0~146.0~152.0~116.0,184.0~114.0,184.0~114.0,11.5~12.600000381469728,1.1,1.1,306.0~330.0,3.7~3.5,3.7~3.5,129.0~131.0~133.0~137.0~137.0,129.0~131.0~133.0~137.0~137.0,12.2~11.1,12.2~11.1,12.2~11.1,23.3,23.3,full code
7,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,F,50.1,25.5,NaN,NaN,17.0~20.0~18.0~20.0~15.0~19.0~17.0~18.0~18.0~2...,86.0~90.0~95.0~97.0~89.0~93.0~80.0~90.0~89.0~8...,NaN,NaN,115.0~112.0~106.0~104.0~100.0~98.0~115.0~107.0...,NaN,58.0~56.0~50.0~42.0~52.0~47.0~61.0~57.0~64.0~7...,NaN,36.88890075683594~36.83330154418945~37.3889007...,36.89~36.83~37.39~36.94~36.33~36.44~36.61~36.72,NaN,96.0~97.0~96.0~96.0~96.0~97.0~93.0~99.0~99.0~9...,4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4.0~4....,5.0~5.0~5.0~5.0~5.0~5.0~5.0~5.0~5.0~5.0~4.0~5....,6.0~6.0~6.0~6.0~6.0~6.0~6.0~6.0~5.0~6.0~5.0~6....,NaN,NaN,NaN,14.0~19.0,14.0~19.0,106.0~106.0,0.7~0.7,0.7~0.7,120.0~149.0~119.0~116.0,129.0~130.0~126.0,129.0~130.0~126.0,12.600000381469728~12.5,NaN,NaN,258.0~243.0,3.9~3.8,3.9~3.8,141.0~142.0,141.0~142.0,9.3~12.8,9.3~12.8,9.3~12.8,NaN,NaN,full code
9,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaT,NaT,CORONARY ARTERY DISEASE,0,F,39.9,6.9,0.0~nan,14.0~28.0~26.0~nan,19.0~15.0~20.0~16.0~19.0~18.0~15.0~12.0~14.0~1...,80.0~77.0~78.0~90.0~92.0~76.0~66.0~60.0~64.0~6...,151.0~147.0~151.0~135.0~127.0~121.0~105.0~114....,NaN,112.0~98.0~117.0~102.0,73.0~72.0~71.0~70.0~60.0~58.0~63.0~59.0~59.0~5...,51.0~39.0~65.0~39.0,36.5~36.59999847412109~36.29999923706055~36.59...,37.27780151367188~36.44440078735352~35.9444007...,37.28~36.44~35.94~36.22~37.94~38.11~37.83~37.22,36.50~36.60~36.30~36.60~36.80~37.10~37.40~37.4...,98.0~98.0~96.0~95.0~95.0~94.0~95.0~96.0~96.0~9...,4.0~4.0~4.0~4.0~4.0~1.0~3.0~3.0~4.0~4.0~4.0~4....,5.0~5.0~5.0~5.0~5.0~1.0~1.0~1.0~1.0~5.0~5.0~5....,6.0~6.0~6.0~6.0~6.0~1.0~4.0~4.0~6.0~6.0~6.0~6....,16.0,16.0,3.9,13.0~18.0~15.0~14.0~16.0~13.0,13.0~18.0~15.0~14.0~16.0~13.0,106.0~108.0~110.0~108.0~104.0,0.6~0.8~0.5~0.6~0.6~0.4,0.6~0.8~0.5~0.6~0.6~0.4,149.0~205.0~109.0~197.0~164.0~137.0,169.0~164.0~197.0~135.0~239.0~154.0~119.0~158....,169.0~164.0~197.0~135.0~239.0~154.0~119.0~158....,12.300000190734863~12.0~10.800000190734863~

In [204]:
df_result7 = df_result6.copy()

for col in list(data_info.keys()):
    range_ = data_info[col][-1]
    for idx, val in df_result7[col].items():
        if pd.isna(val):
            continue
        elif str(val).lower() == 'nan':
            df_result7.at[idx, col] = pd.NA
        try:
            num = float(val)
            if num < range_[0] or num > range_[1]:
                df_result7.at[idx, col] = pd.NA
            else:
                df_result7.at[idx, col] = num
        except ValueError:
            if '~' in str(val):
                parts = str(val).split('~')
                valid_parts = []
                for part in parts:
                    try:
                        num = float(part)
                        if range_[0] <= num <= range_[1]:
                            valid_parts.append(num)
                    except ValueError:
                        continue
                if valid_parts:
                    df_result7.at[idx, col] = sum(valid_parts) / len(valid_parts)
                else:
                    df_result7.at[idx, col] = pd.NA
            else:
                print(f"Invalid string '{val}' at row {idx} in column {col}")
                df_result7.at[idx, col] = pd.NA
df_result7

Invalid string '' at row 6199 in column Body temperature
Invalid string '' at row 8397 in column Body temperature
Invalid string '' at row 8467 in column Body temperature
Invalid string '' at row 10748 in column Body temperature
Invalid string '' at row 10957 in column Body temperature
Invalid string '' at row 11042 in column Body temperature
Invalid string '' at row 11203 in column Body temperature
Invalid string '' at row 12629 in column Body temperature
Invalid string '' at row 14400 in column Body temperature
Invalid string '' at row 15660 in column Body temperature
Invalid string '' at row 18848 in column Body temperature
Invalid string '' at row 19462 in column Body temperature
Invalid string '' at row 24413 in column Body temperature
Invalid string '<4' at row 22153 in column Alanine aminotransferase
Invalid string 'no data' at row 11401 in column Aspartate aminotransferase
Invalid string '<1.0*' at row 4623 in column Albumin
Invalid string '<1.0*' at row 5122 in column Albumin


,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,Respiratory rate,Heart rate,Systolic blood pressure,Diastolic blood pressure,Body temperature,Saturation of peripheral oxygen,Glasgow coma scale - eye,Glasgow coma scale - verbal,Glasgow coma scale - motor,Alanine aminotransferase,Aspartate aminotransferase,Albumin,Blood urea nitrogen,Bilirubin,C-reactive protein,Chloride,Creatinine,Glucose,Hemoglobin,Prothrombin time (INR),Platelets,Potassium,Sodium,White blood cell,Activated partial thromboplastin time
5,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,M,41.8,4.9,17.542857,83.771429,133.588235,71.0,36.096,98.285714,1.0,1.0,1.083333,NaN,NaN,NaN,30.5,<NA>,<NA>,102.5,1.85,149.0,12.05,1.1,318.0,3.6,133.4,11.65,23.3
7,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,F,50.1,25.5,18.176471,84.918919,106.666667,54.194444,36.76875,96.655172,4.0,4.941176,5.882353,NaN,NaN,NaN,16.5,<NA>,<NA>,106.0,0.7,128.333333,12.55,NaN,250.5,3.85,141.5,11.05,NaN
9,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaT,NaT,CORONARY ARTERY DISEASE,0,F,39.9,6.9,17.646552,98.017241,107.25,48.5,37.1225,98.59292,3.782609,4.304348,5.608696,16.0,16.0,3.9,14.833333,<NA>,<NA>,107.2,0.583333,158.833333,10.82,1.4,135.5,4.175,138.375,17.2,33.875
11,19,109235,2108-08-05 16:25:00,2108-08-11 11:29:00,2108-08-05 08:01:00,2108-08-05 20:12:00,C-2 FRACTURE,0,M,300.0,5.8,20.583333,98.416667,146.708333,62.0,36.872857,98.208333,4.0,4.2,6.0,NaN,NaN,NaN,25.0,<NA>,<NA>,105.0,1.2,139.0,12.8,1.3,216.0,4.2,138.0,10.1,30.4
12,20,157681,2183-04-28 09:45:00,2183-05-03 14:45:00,NaT,NaT,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,F,75.9,5.2,15.512821,79.121951,118.333333,42.166667,36.0,98.435897,3.0,3.0,4.333333,NaN,NaN,NaN,18.0,<NA>,<NA>,108.0,0.8,153.75,9.5,1.45,121.5,4.088889,137.5,17.5,32.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26366,32786,165351,2152-11-15 20:26:00,2152-12-08 11:00:00,2152-11-15 18:39:00,2152-11-15 22:21:00,SUBARACHNOID HEMORRHAGE,0,M,53.7,22.6,22.461314,68.940666,152.684211,55.041509,37.95435,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,<NA>,<NA>,105.275862,0.927586,161.782609,10.844444,1.1,448.740741,4.330303,139.896552,16.648148,26.382609
26367,32786,163011,2152-12-09 19:17:00,2152-12-15 14:30:00,2152-12-09 15:28:00,2152-12-09 22:54:00,ALTERED MENTAL STATUS,0,M,53.8,5.8,23.518519,61.888889,131.423077,66.192308,37.23,96.923077,2.0,1.0,1.857143,475.0,269.5,3.7,82.0,<NA>,<NA>,107.0,1.35,159.0,10.8,1.2,457.0,4.65,147.0,14.15,25.7
26368,32787,157308,2102-10-02 07:15:00,2102-10-09 16:42:00,NaT,NaT,AORTIC STENOSIS\AORTIC AND MITRAL VALVE REPLAC...,0,M,44.7,7.4,16.694737,81.526316,115.5,71.904762,37.5,97.543478,3.411765,4.058824,5.647059,NaN,NaN,NaN,17.0,<NA>,<NA>,106.0,1.15,116.473684,10.7,1.2,174.4,4.041667,139.6,14.22,40.433333
26376,32807,140728,2173-10-27 13:00:00,2173-11-02 13:50:00,NaT,NaT,CORONARY ARTERY DISEASE,0,M,81.0,6.0,16.178082,75.05,128.53125,69.875,35.776,98.2875,3.461538,3.153846,5.615385,18.0,40.0,3.9,22.25,<NA>,<NA>,102.5,1.25,107.133333,10.84,1.15,158.0,4.49375,134.0,8.38,52.316667


In [206]:
df_result7.to_csv("MIMIC3_CC_v1.csv",index=False)

In [207]:
df_result7.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14404 entries, 5 to 26377
Data columns (total 36 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   subject_id                             14404 non-null  int64         
 1   hadm_id                                14404 non-null  int64         
 2   admittime                              14404 non-null  datetime64[ns]
 3   dischtime                              14404 non-null  datetime64[ns]
 4   edregtime                              7279 non-null   datetime64[ns]
 5   edouttime                              7279 non-null   datetime64[ns]
 6   diagnosis                              14404 non-null  object        
 7   hospital_expire_flag                   14404 non-null  int64         
 8   gender                                 14404 non-null  object        
 9   age                                    14404 non-null  float64    